###  Computations for the paper: ###

## "Drinfeld modular polynomials of level $T$" ##
by Florian Breuer and Mahefason Heriniaina Razafinjatovo

Since I'm not very proficient in Pari/GP, but computations in the GP calculator are way faster than anything I can do with Sage,
this notebook contains Sage code that produces Pari code (as a string), which you can then copy & paste into the GP calculator.
<hr>


### Case 1: Outgoing isogenies. ###
We want to compute the polynomial
$$\Phi_{J, (A/TA)}(X)$$
The following code cell produces Pari/GP output.

In [3]:
# Main parameters - you can play with these:
q = 2
r = 3
elist = [4,1]   # defines the invariant J = g_1^e_1...g_{r-1}^e_{r-1}

# psi_t(r):
d = (q^r-1)/(q-1)

# define the variables
T = var('T')
x = var('x')
u = [var(f'u{i}') for i in range(1, r)]

fieldvars = [T] + u

# The field
A = GF(q)[fieldvars]
F = A.fraction_field()

for v in fieldvars:
    v = F(v)

# characteristic polynomial
P = x^d + (-1)^r*T + sum((-1)^i*u[r-i-1]*x^(d - (q^r - q^(r-i))/(q-1)) for i in range(1,r))

# the matrix:
def matrix_with_characteristic_polynomial(p):
    """
    Construct a matrix with the given characteristic polynomial.
    
    :param p: The characteristic polynomial (a SageMath polynomial).
    :return: A square matrix with the given characteristic polynomial.
    """
    n = p.degree(x)
    coeffs = p.coefficients(x, sparse=False)[:-1]  # Exclude the leading coefficient (monic polynomial)
    return matrix(F, n, lambda i, j: 1 if i - 1 == j else -coeffs[i] if j == n - 1 else 0)

V = matrix_with_characteristic_polynomial(P)

# code string for the matrix
def gp_matrix(M, d, q):
    '''Returns a string to define the matrix M mod q in GP'''
    s = 'V = Mod(['
    s += '; '.join(','.join(str(M[i][j]) for j in range(d)) for i in range(d))
    s += f'], {q});\n'
    return s

# code string for variable definitions
def gp_vars(fieldvars):
    '''Returns a string to define the variables in GP'''
    s = '\n'.join(f"{v} = Pol([1,0], '{v});" for v in fieldvars) + '\n'
    return s

# code string for the definition of the g matrices
def gp_g():
    '''Returns a string to define the matrices g1, g2, ...'''
    s = f"g1 = W^{q}*(V*u1 + (T^{q} - T)*Id);\n"
    for k in range(2,r):
        s += f"g{k} = W^{q^k}*(V*u{k} + u{k-1}^{q}*Id - g{k-1});\n"
    return s

# code string for J
def gp_J(*elist):
    '''Return a string to define the matrix J.
    elist is a list of the weights e1, e2, ...'''
    s = "J = " + '*'.join(f"g{k+1}^{elist[k]}" for k in range(r-1)) + ';\n'
    return s

# compute the weight, output code for P and list of degrees
def weightJ(*elist):
    w1 = sum((q^i-1)*elist[i-1] for i in range(1, r))
    if not w1%(q^r-1) == 0:
        raise ValueError(f'Invalid invariant: {w1} not divisible by {q^r-1}')
    er = w1/(q^r-1)
    return q*(sum(elist[i-1] for i in range(1, r)) - er)

# Now put everything together!
s = gp_vars(fieldvars) + gp_matrix(V, d, q)
s += f"W = V^(-1);\nId = Mod(matid({d}), {q});\n"
s += gp_g() + gp_J(*elist)
s += f"w = {weightJ(*elist)};\n"

s += f'''P = charpoly(J);
         
for (i=0, {d-1}, coeff_x = polcoeff(P, i, 'x); deg_T = poldegree(coeff_x, 'T); print(i, " & ", deg_T, " & ", w*({d}-i)) );'''

print(s)

T = Pol([1,0], 'T);
u1 = Pol([1,0], 'u1);
u2 = Pol([1,0], 'u2);
V = Mod([0,0,0,0,0,0,T; 1,0,0,0,0,0,u1; 0,1,0,0,0,0,0; 0,0,1,0,0,0,u2; 0,0,0,1,0,0,0; 0,0,0,0,1,0,0; 0,0,0,0,0,1,0], 2);
W = V^(-1);
Id = Mod(matid(7), 2);
g1 = W^2*(V*u1 + (T^2 - T)*Id);
g2 = W^4*(V*u2 + u1^2*Id - g1);
J = g1^4*g2^1;
w = 8;
P = charpoly(J);
         
for (i=0, 6, coeff_x = polcoeff(P, i, 'x); deg_T = poldegree(coeff_x, 'T); print(i, " & ", deg_T, " & ", w*(7-i)) );


 You can copy & paste it directly into the GP calculator, or into the gp magic cell below (slower):

In [4]:
%%gp 

T = Pol([1,0], 'T);
u1 = Pol([1,0], 'u1);
u2 = Pol([1,0], 'u2);
V = Mod([0,0,0,0,0,0,T; 1,0,0,0,0,0,u1; 0,1,0,0,0,0,0; 0,0,1,0,0,0,u2; 0,0,0,1,0,0,0; 0,0,0,0,1,0,0; 0,0,0,0,0,1,0], 2);
W = V^(-1);
Id = Mod(matid(7), 2);
g1 = W^2*(V*u1 + (T^2 - T)*Id);
g2 = W^4*(V*u2 + u1^2*Id - g1);
J = g1^4*g2^1;
w = 8;
P = charpoly(J);
         
for (i=0, 6, coeff_x = polcoeff(P, i, 'x); deg_T = poldegree(coeff_x, 'T); print(i, " & ", deg_T, " & ", w*(7-i)) );













0 & 56 & 56
1 & 48 & 48
2 & 40 & 40
3 & 32 & 32
4 & 24 & 24
5 & 16 & 16
6 & 8 & 8


### Case 2: Incoming isogenies. ###
We want to compute the polynomial
$$\Phi_{J, (A/TA)^{r-1}}(X)$$
The following code cell produces Pari/GP output.

In [5]:
# Main parameters, you can change these:
q = 2
r = 3
elist = [4,1]   # defines the invariant J = g_1^e_1...g_{r-1}^e_{r-1}

# psi_t(r):
d = (q^r-1)/(q-1)

# define the variables
T = var('T')
x = var('x')
u = [var(f'u{i}') for i in range(1, r)]

fieldvars = [T] + u

# The field
A = GF(q)[fieldvars]
F = A.fraction_field()

for v in fieldvars:
    v = F(v)

# characteristic polynomial
P = x^d + (-1)^r*T^(q^(r-1)) + sum((-1)^i*u[r-i-1]^(q^(i-1))*x^(d - (q^i - 1)/(q-1)) for i in range(1,r))

# the matrix:
def matrix_with_characteristic_polynomial(p):
    """
    Construct a matrix with the given characteristic polynomial.
    
    :param p: The characteristic polynomial (a SageMath polynomial).
    :return: A square matrix with the given characteristic polynomial.
    """
    n = p.degree(x)
    coeffs = p.coefficients(x, sparse=False)[:-1]  # Exclude the leading coefficient (monic polynomial)
    return matrix(F, n, lambda i, j: 1 if i - 1 == j else -coeffs[i] if j == n - 1 else 0)

V = matrix_with_characteristic_polynomial(P)

# code string for the definition of the g matrices
def gp_g2():
    '''Returns a string to define the matrices g1, g2, ...'''
    s = f"g1 = W*(T - T^{q}) + V^{q-1}*u1;\n"
    for k in range(2,r):
        s += f"g{k} = W*(u{k-1}*Id - g{k-1}^{q}) + V^{q^k-1}*u{k};\n"
    return s

# compute the weight, output code for P and list of degrees
def weightJ2(*elist):
    w1 = sum((q^i-1)*elist[i-1] for i in range(1, r))
    if not w1%(q^r-1) == 0:
        raise ValueError(f'Invalid invariant: {w1} not divisible by {q^r-1}')
    er = w1/(q^r-1)
    return sum(elist[i-1] for i in range(1, r)) + er*(q^r - q^(r-1) -1)


s = gp_vars(fieldvars) + gp_matrix(V, d, q)
s += f"W = V^(-1);\nId = Mod(matid({d}), {q});\n"
s += gp_g2() + gp_J(*elist)
s += f"w = {weightJ2(*elist)};\n"

s += f'''P = charpoly(J);
         
for (i=0, {d-1}, coeff_x = polcoeff(P, i, 'x); deg_T = poldegree(coeff_x, 'T); print(i, " & ", deg_T, " & ", w*({d}-i)) );'''

print(s)

T = Pol([1,0], 'T);
u1 = Pol([1,0], 'u1);
u2 = Pol([1,0], 'u2);
V = Mod([0,0,0,0,0,0,T^4; 1,0,0,0,0,0,0; 0,1,0,0,0,0,0; 0,0,1,0,0,0,0; 0,0,0,1,0,0,u1^2; 0,0,0,0,1,0,0; 0,0,0,0,0,1,u2], 2);
W = V^(-1);
Id = Mod(matid(7), 2);
g1 = W*(T - T^2) + V^1*u1;
g2 = W*(u1*Id - g1^2) + V^3*u2;
J = g1^4*g2^1;
w = 8;
P = charpoly(J);
         
for (i=0, 6, coeff_x = polcoeff(P, i, 'x); deg_T = poldegree(coeff_x, 'T); print(i, " & ", deg_T, " & ", w*(7-i)) );


In [6]:
%%gp 

T = Pol([1,0], 'T);
u1 = Pol([1,0], 'u1);
u2 = Pol([1,0], 'u2);
V = Mod([0,0,0,0,0,0,T^4; 1,0,0,0,0,0,0; 0,1,0,0,0,0,0; 0,0,1,0,0,0,0; 0,0,0,1,0,0,u1^2; 0,0,0,0,1,0,0; 0,0,0,0,0,1,u2], 2);
W = V^(-1);
Id = Mod(matid(7), 2);
g1 = W*(T - T^2) + V^1*u1;
g2 = W*(u1*Id - g1^2) + V^3*u2;
J = g1^4*g2^1;
w = 8;
P = charpoly(J);
         
for (i=0, 6, coeff_x = polcoeff(P, i, 'x); deg_T = poldegree(coeff_x, 'T); print(i, " & ", deg_T, " & ", w*(7-i)) );













0 & 56 & 56
1 & 48 & 48
2 & 40 & 40
3 & 32 & 32
4 & 24 & 24
5 & 16 & 16
6 & 8 & 8
